<a href="https://colab.research.google.com/github/yongsuk-lee-korea/insurance-house/blob/main/Llama3_%ED%8C%8C%EC%9D%B8%ED%8A%9C%EB%8B%9D%EC%9A%A9_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **환경설정 (본인 구글드라이브 내 dataset 폴더**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
dataPath = '/content/gdrive/MyDrive/Colab Notebooks/llama2_custom/dataset/'

In [ ]:
!pip install jsonlines

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
import pandas as pd
import json

In [ ]:
import jsonlines

In [ ]:
!pip install datasets==2.19.0

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset

In [ ]:
!pip install gspread gspread_dataframe


In [ ]:
#데이터 경로 및 설정 불러오기
datasetName = "indata_eng.csv"
jsonFileName = "indata_eng.jsonl"

In [ ]:
def csv_to_json(csv_file_path, json_file_path):
    try:
        df = pd.read_csv(csv_file_path, encoding='utf-8')  # 인코딩을 명시적으로 지정
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file_path, encoding='cp949')  # 다른 인코딩을 시도
    with open(json_file_path, "w", encoding='utf-8') as json_file:
        for index, row in df.iterrows():
            data = {'Inputs': row['inputs'], 'response': row['response']}
            json.dump(data, json_file, ensure_ascii=False)
            json_file.write("\n")

csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName

csv_to_json(csv_file_path, json_file_path)


In [ ]:
!pip install gspread gspread_dataframe


**파인튜닝용 포맷변환(json)**

In [ ]:
import jsonlines
from datasets import Dataset

# JSONLines 파일을 읽어서 데이터셋 생성
indataset = []
with jsonlines.open(json_file_path) as f:
    for line in f.iter():
        indataset.append(f"<s>[INST] {line['Inputs']} [/INST] {line['response']} </s>")

print('데이터셋 확인')
print(indataset[:5])

# 데이터셋을 사전으로 변환하여 Hugging Face Dataset으로 저장
indataset = Dataset.from_dict({'text': indataset})
indataset.save_to_disk(dataPath)

print('데이터셋 info 확인')
print(indataset)


데이터셋 확인
['<s>[INST] What does hkcode YouTube teach? [/INST] We are sharing basic learning content for easy access to big data artificial intelligence on the hkcode YouTube channel. </s>', '<s>[INST] What does the Smart Finance Department mainly teach? [/INST] The Smart Finance Department teaches big data, artificial intelligence, web development, and blockchain. </s>', '<s>[INST] How much does admission cost? [/INST] It is a government-funded course and is free. </s>', '<s>[INST] How many people does the Smart Finance Department recruit per year? [/INST] We are recruiting 27 people per year </s>', '<s>[INST] What is the average employment rate? [/INST] The recent average three-year employment rate is over 95%. </s>']


Saving the dataset (0/1 shards):   0%|          | 0/39 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 39
})


In [ ]:
indataset

Dataset({
    features: ['text'],
    num_rows: 39
})

In [ ]:
# 허깅페이스에 업로드
indataset.push_to_hub("kch8909/llama3_movie")